### Cost model benchmarking - case study #1

Loosely based on Dudgeon Wind Farm (400 MW, 8MW turbines)

National Renewable Energy Lab

Matt Shields, Jake Nunemaker

Updated: 2/17/2020

In [1]:
import os
import sys
import pprint
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as plt_dates

import ORBIT
print(f"Using ORBIT version {ORBIT.__version__}.")
from ORBIT import ProjectManager


usd_to_euro = 0.92

Using ORBIT version 0+unknown.


#### Weather File

In [2]:
filepath = os.path.join('library', 'weather', 'dudgeon_wave_100m_wind.csv')
weather = pd.read_csv(filepath).set_index(keys='datetime')
weather.describe()

,windspeed,waveheight
count,20616.000000,20616.000000
mean,9.138532,1.183350
std,4.249767,0.667603
min,0.121744,0.170372
25%,5.980583,0.698537
50%,8.916669,1.027665
75%,11.878717,1.507961
max,25.680700,5.773216


#### ORBIT Configuration

In [3]:
phases = [
    # Substructures
    'MonopileInstallation',
#     'ScourProtectionDesign',
#     'ScourProtectionInstallation',
    
    # Turbines
    'TurbineInstallation',
    
#     # Electrical
    'ArraySystemDesign',
    'ArrayCableInstallation',
    'ExportSystemDesign',
    'ExportCableInstallation',
#     'OffshoreSubstationInstallation'
]

required_config = ProjectManager.compile_input_dict(phases)
# required_config

In [4]:
config = {
    # Substations
#     "num_substations": 1,
    
    # Vessels
#     'scour_protection_install_vessel': 'ExampleScour',
#     'trench_dig_vessel': 'StematSpirit',
#     'array_cable_lay_vessel': 'cable_lay_vessel',
#     'export_cable_lay_vessel': 'StematSpirit',
#     "oss_install_vessel": "OlegStrashnov",  # Actually vessel from SPT Offshore
    
    # Site/plant
    'site': {
        'depth': 22.5,
        'distance': 124,
        'distance_to_landfall': 42,
        'distance_to_beach': 0,
        'distance_to_interconnection': 3,
        'mean_windspeed': 9.13
    },
    
    'plant': {
        'layout': 'grid',
        'num_turbines': 50,
        'row_spacing': 7,
        'turbine_spacing': 9,
        'substation_distance': 1
    },
    
    'port': {
        'num_cranes': 1,
        'monthly_rate': 2000000,
        "name": "Green Port"
    },
    
    # Turbine + components
    'turbine': '8MW_generic',

    # Substructure components
    'substructure': {'diameter': 7.2},
    'monopile': {
        'type': 'Monopile',
        'length': 69,
        'diameter': 7.2,
        'deck_space': 496.8,
        'mass': 800,
        'monopile_steel_cost' : 2000 / usd_to_euro,
    },
    
    'transition_piece': {
        'type': 'Transition Piece',
        'deck_space': 100,
        'mass': 400,
        'transition_piece_steel_cost' : 2000 / usd_to_euro,
    },
    
#     'scour_protection_design': {
#         'cost_per_tonne': 40,
#     },
    
    # Electrical
    'array_system_design': {
        'cables': 'XLPE_630mm_66kV'
    },
    
    'array_system': {
        'strategy': 'lay_bury'
    },
    
    'export_system': {
        'strategy': 'lay_bury'
    },
    
    'export_system_design': {
        'cables': 'XLPE_1000mm_220kV',
        'percent_added_length': 0
    },
    
#     "offshore_substation_topside": {
#         "type": "Topside",
#         "deck_space": 200,
#         "weight": 2000,
#     },
#     "offshore_substation_substructure": {
#         "type": "Monopile",
#         "deck_space": 500,
#         "weight": 1850,
#         "length": 69,  # Assumed to be the same as monopile length
#     },


    # Phase specific configurations
    'MonopileInstallation': {
        'wtiv': 'Benchmarking_WTIV_turbine',  
        
    },
    
    'TurbineInstallation': {
        'wtiv': 'Benchmarking_WTIV_turbine'
    },
    
    'ArrayCableInstallation': {
        'array_cable_install_vessel' : 'cable_lay_vessel'
    }, 
    
    'ExportCableInstallation': {
        'export_cable_install_vessel' : 'cable_lay_vessel'
    },
    
#     'OffshoreSubstationInstallation': {
#         "num_feeders": 1,
#         "feeder": "seajacks_scylla",
#     },    
    
    # Phases
    'design_phases': [
#         "MonopileDesign",
#         'ScourProtectionDesign',
        'ArraySystemDesign',
        'ExportSystemDesign',
#         'OffshoreSubstationDesign'
    ],
    
    'install_phases': {
        'MonopileInstallation': '07/01/2016',         # Updated dates 
#         'ScourProtectionInstallation': '09/01/2016',  # Placed at the end of the monopile installation
        'TurbineInstallation': '07/01/2016',
        'ArrayCableInstallation': '07/01/2016',
        'ExportCableInstallation': '07/01/2016',
#         'OffshoreSubstationInstallation': '08/01/2016'
    }
}

In [5]:
path = os.path.join(os.getcwd(), "library")
project = ProjectManager(config, weather=weather, library_path=path)

ORBIT library intialized at 'C:\Users\mshields\Documents\Analysis Tools\NREL\ORBIT\Analysis\library'


In [6]:
project.run_project()

# Results

### Turbine installation

In [7]:
# Trip data
project._phases['TurbineInstallation'].wtiv.trip_data

[Trip(cargo_mass=3012, deck_space=3884, items={TowerSection: 8, Nacelle: 4, Blade: 12}),
 Trip(cargo_mass=3012, deck_space=3884, items={TowerSection: 8, Nacelle: 4, Blade: 12}),
 Trip(cargo_mass=3012, deck_space=3884, items={TowerSection: 8, Nacelle: 4, Blade: 12}),
 Trip(cargo_mass=3012, deck_space=3884, items={TowerSection: 8, Nacelle: 4, Blade: 12}),
 Trip(cargo_mass=3012, deck_space=3884, items={TowerSection: 8, Nacelle: 4, Blade: 12}),
 Trip(cargo_mass=3012, deck_space=3884, items={TowerSection: 8, Nacelle: 4, Blade: 12}),
 Trip(cargo_mass=3012, deck_space=3884, items={TowerSection: 8, Nacelle: 4, Blade: 12}),
 Trip(cargo_mass=3012, deck_space=3884, items={TowerSection: 8, Nacelle: 4, Blade: 12}),
 Trip(cargo_mass=3012, deck_space=3884, items={TowerSection: 8, Nacelle: 4, Blade: 12}),
 Trip(cargo_mass=3012, deck_space=3884, items={TowerSection: 8, Nacelle: 4, Blade: 12}),
 Trip(cargo_mass=3012, deck_space=3884, items={TowerSection: 8, Nacelle: 4, Blade: 12}),
 Trip(cargo_mass=3012

In [8]:
# Installation costs 
turb_time = np.round(project._phases['TurbineInstallation'].total_phase_time * (1/24) ,1 ) # Convert hours to days
turb_cost = np.round(project._phases['TurbineInstallation'].total_phase_cost * 1e-6 * usd_to_euro, 1)

print('Turbine install time: ', turb_time, 'days')
print('Turbine install cost: ', turb_cost, 'M Euros')

Turbine install time:  149.1 days
Turbine install cost:  38.8 M Euros


In [9]:
proj_df = pd.DataFrame(project.project_actions)
proj_df.loc[proj_df['phase']=='TurbineInstallation']

,action,agent,cable_lay_speed,cost,cost_multiplier,duration,hub_height,level,location,max_waveheight,max_windspeed,phase,phase_name,site_depth,time,transit_speed
1,Mobilize,WTIV,NaN,322500.000000,0.5,72.000000,NaN,ACTION,NaN,NaN,NaN,TurbineInstallation,NaN,NaN,0.000000,NaN
5,Fasten Tower Section,WTIV,NaN,35833.333333,NaN,4.000000,113.0,ACTION,NaN,NaN,NaN,TurbineInstallation,TurbineInstallation,22.5,4.000000,NaN
11,Fasten Tower Section,WTIV,NaN,35833.333333,NaN,4.000000,113.0,ACTION,NaN,NaN,NaN,TurbineInstallation,TurbineInstallation,22.5,8.000000,NaN
13,Fasten Nacelle,WTIV,NaN,35833.333333,NaN,4.000000,113.0,ACTION,NaN,NaN,NaN,TurbineInstallation,TurbineInstallation,22.5,12.000000,NaN
16,Fasten Blade,WTIV,NaN,13437.500000,NaN,1.500000,113.0,ACTION,NaN,NaN,NaN,TurbineInstallation,TurbineInstallation,22.5,13.500000,NaN
18,Fasten Blade,WTIV,NaN,13437.500000,NaN,1.500000,113.0,ACTION,NaN,NaN,NaN,TurbineInstallation,TurbineInstallation,22.5,15.000000,NaN
20,Fasten Blade,WTIV,NaN,13437.500000,NaN,1.500000,113.0,ACTION,NaN,NaN,NaN,TurbineInstallation,TurbineInstallation,22.5,16.500000,NaN
24,Fasten Tower Section,WTIV,NaN,35833.333333,NaN,4.000000,113.0,ACTION,NaN,NaN,NaN,TurbineInstallation,TurbineInstallation,22.5,20.500000,NaN
26,Fasten Tower Section,WTIV,NaN,35833.333333,NaN,4.000000,113.0,ACTION,NaN,NaN,NaN,TurbineInstallation,TurbineInstallation,22.5,24.500000,NaN
29,Fasten Nacelle,WTIV,NaN,35833.333333,NaN,4.000000,113.0,ACTION,NaN,NaN,NaN,TurbineInstallation,TurbineInstallation,22.5,28.500000,NaN


### Monopile design and installation

In [10]:
# Trip data
project._phases['MonopileInstallation'].wtiv.trip_data

[Trip(cargo_mass=7200, deck_space=3580.8, items={Monopile: 6, TransitionPiece: 6}),
 Trip(cargo_mass=7200, deck_space=3580.8, items={Monopile: 6, TransitionPiece: 6}),
 Trip(cargo_mass=7200, deck_space=3580.8, items={Monopile: 6, TransitionPiece: 6}),
 Trip(cargo_mass=7200, deck_space=3580.8, items={Monopile: 6, TransitionPiece: 6}),
 Trip(cargo_mass=7200, deck_space=3580.8, items={Monopile: 6, TransitionPiece: 6}),
 Trip(cargo_mass=7200, deck_space=3580.8, items={Monopile: 6, TransitionPiece: 6}),
 Trip(cargo_mass=7200, deck_space=3580.8, items={Monopile: 6, TransitionPiece: 6}),
 Trip(cargo_mass=7200, deck_space=3580.8, items={Monopile: 6, TransitionPiece: 6}),
 Trip(cargo_mass=2400, deck_space=1193.6, items={Monopile: 2, TransitionPiece: 2})]

In [11]:
# Installation and capital costs 
mono_time = np.round(project._phases['MonopileInstallation'].total_phase_time * (1/24) , 1) # Convert hours to days
mono_install_cost = np.round(project._phases['MonopileInstallation'].total_phase_cost * 1e-6 * usd_to_euro, 1)
mono_capital_cost = np.round(((config['monopile']['mass'] * 
                     config['plant']['num_turbines'] * 
                     config['monopile']['monopile_steel_cost']) +
                     (config['transition_piece']['mass'] * 
                     config['plant']['num_turbines'] * 
                     config['transition_piece']['transition_piece_steel_cost']))* 1e-6 * usd_to_euro , 1)

print('Monopile install time: ', mono_time, 'days')
print('Monopile install cost: ', mono_install_cost, 'M Euros')
print('Monopile capital cost: ', mono_capital_cost, 'M Euros')
print('Monopile total cost: ', mono_capital_cost + mono_install_cost, 'M Euros')

Monopile install time:  87.4 days
Monopile install cost:  22.9 M Euros
Monopile capital cost:  120.0 M Euros
Monopile total cost:  142.9 M Euros


In [12]:
proj_df = pd.DataFrame(project.project_actions)
proj_df.loc[proj_df['phase']=='MonopileInstallation']

,action,agent,cable_lay_speed,cost,cost_multiplier,duration,hub_height,level,location,max_waveheight,max_windspeed,phase,phase_name,site_depth,time,transit_speed
0,Mobilize,WTIV,NaN,322500.000000,0.5,72.000000,NaN,ACTION,NaN,NaN,NaN,MonopileInstallation,NaN,NaN,0.000000,NaN
12,Fasten Monopile,WTIV,NaN,107500.000000,NaN,12.000000,113.0,ACTION,NaN,NaN,NaN,MonopileInstallation,MonopileInstallation,22.5,12.000000,NaN
23,Fasten Transition Piece,WTIV,NaN,71666.666667,NaN,8.000000,113.0,ACTION,NaN,NaN,NaN,MonopileInstallation,MonopileInstallation,22.5,20.000000,NaN
32,Fasten Monopile,WTIV,NaN,107500.000000,NaN,12.000000,113.0,ACTION,NaN,NaN,NaN,MonopileInstallation,MonopileInstallation,22.5,32.000000,NaN
37,Fasten Transition Piece,WTIV,NaN,71666.666667,NaN,8.000000,113.0,ACTION,NaN,NaN,NaN,MonopileInstallation,MonopileInstallation,22.5,40.000000,NaN
46,Fasten Monopile,WTIV,NaN,107500.000000,NaN,12.000000,113.0,ACTION,NaN,NaN,NaN,MonopileInstallation,MonopileInstallation,22.5,52.000000,NaN
51,Fasten Transition Piece,WTIV,NaN,71666.666667,NaN,8.000000,113.0,ACTION,NaN,NaN,NaN,MonopileInstallation,MonopileInstallation,22.5,60.000000,NaN
58,Fasten Monopile,WTIV,NaN,107500.000000,NaN,12.000000,113.0,ACTION,NaN,NaN,NaN,MonopileInstallation,MonopileInstallation,22.5,72.000000,NaN
64,Fasten Transition Piece,WTIV,NaN,71666.666667,NaN,8.000000,113.0,ACTION,NaN,NaN,NaN,MonopileInstallation,MonopileInstallation,22.5,80.000000,NaN
74,Fasten Monopile,WTIV,NaN,107500.000000,NaN,12.000000,113.0,ACTION,NaN,NaN,NaN,MonopileInstallation,MonopileInstallation,22.5,92.000000,NaN


### Array cable design and installation

In [13]:
# Trip data
# project._phases['ArrayCableInstallation'].install_vessel.__dict__
# project._phases.keys()
# project._phases['ArraySystemDesign'].__dict__

project._phases['ArraySystemDesign'].detailed_output

{'array_system': {'cables': {'XLPE_630mm_66kV': {'cable_sections': [(2.957580471,
      2),
     (1.6199999999999999, 46),
     (2.0403069939000003, 2)],
    'linear_density': 45}}},
 'array_system_num_strings': 5,
 'array_system_total_length': 84.5157749298,
 'array_system_length_by_type': {'XLPE_630mm_66kV': 84.5157749298},
 'array_system_total_cost': 34397920.3964286,
 'array_system_cost_by_type': {'XLPE_630mm_66kV': 34397920.3964286}}

In [14]:
# Installation and capital costs 
ac_time = np.round(project._phases['ArrayCableInstallation'].total_phase_time * (1/24) , 1) # Convert hours to days
ac_install_cost = np.round(project._phases['ArrayCableInstallation'].total_phase_cost * 1e-6 * usd_to_euro, 1)
ac_capital_cost = np.round(project._phases['ArraySystemDesign'].total_phase_cost * 1e-6 * usd_to_euro, 1)
total_length = np.round(project._phases['ArraySystemDesign'].detailed_output['array_system_length_by_type']['XLPE_630mm_66kV'], 1)

print('Array cable total length: ',total_length , 'km')
print('Array cable install time: ', ac_time, 'days')
print('Array cable install cost: ', ac_install_cost, 'M Euros')
print('Array cable capital cost: ', ac_capital_cost, 'M Euros')
print('Array cable total cost: ', ac_capital_cost + ac_install_cost, 'M Euros')

Array cable total length:  84.5 km
Array cable install time:  69.6 days
Array cable install cost:  13.5 M Euros
Array cable capital cost:  31.6 M Euros
Array cable total cost:  45.1 M Euros


### Export cable 

In [16]:
project._phases['ExportSystemDesign'].detailed_output

{'export_system': {'interconnection_distance': 3,
  'cable': {'linear_density': 80,
   'sections': [45.0225],
   'number': 2,
   'cable_power': 342.9460597040872}},
 'export_system_total_mass': 7203.6,
 'export_system_total_length': 90.045,
 'export_system_total_cost': 53756865.0,
 'export_system_cable_power': 342.9460597040872}

In [30]:
# Installation and capital costs 
ec_time = np.round(project._phases['ExportCableInstallation'].total_phase_time * (1/24) , 1) # Convert hours to days
ec_install_cost = np.round(project._phases['ExportCableInstallation'].total_phase_cost * 1e-6 * usd_to_euro, 1)
ec_capital_cost = np.round(project._phases['ExportSystemDesign'].total_phase_cost * 1e-6 * usd_to_euro, 1)
ec_total_length = np.round(project._phases['ExportSystemDesign'].detailed_output['export_system_total_length'], 1)

print('Export cable total length: ',ec_total_length , 'km')
print('Export cable install time: ', ec_time, 'days')
print('Export cable install cost: ', ec_install_cost, 'M Euros')
print('Export cable capital cost: ', ec_capital_cost, 'M Euros')
print('Export cable total cost: ', ec_capital_cost + ec_install_cost, 'M Euros')

Export cable total length:  90.0 km
Export cable install time:  15.7 days
Export cable install cost:  20.5 M Euros
Export cable capital cost:  49.5 M Euros
Export cable total cost:  70.0 M Euros


### Results - Old

#### Array System Design

In [17]:
project._phases["ArraySystemDesign"].total_cable_length_by_type

{'XLPE_630mm_66kV': 84.5157749298}

In [18]:
project._phases['ArraySystemDesign'].num_full_strings

4

In [19]:
project._phases['ArraySystemDesign'].num_partial_strings

1

In [20]:
# Module outputs
project._phases['ArraySystemDesign'].total_phase_cost

34397920.3964286

In [21]:
total = sum([v for k, v in project._phases['ArraySystemDesign'].total_cable_length_by_type.items()])
print(f"Total length: {total}")

Total length: 84.5157749298


In [22]:
cables = project._phases['ArraySystemDesign'].cables

total_mass = sum([
    cables[k].linear_density*v for k, v in project._phases['ArraySystemDesign'].total_cable_length_by_type.items()
])
total_mass

3803.209871841

#### Export System Design

In [23]:
project._phases['ExportSystemDesign'].total_mass

7203.6

In [24]:
project._phases['ExportSystemDesign'].num_cables

2

In [25]:
project._phases['ExportSystemDesign'].cable.cable_power

342.9460597040872

In [26]:
project._phases['ExportSystemDesign']._plant_capacity

400

In [27]:
project._phases['ExportSystemDesign'].total_phase_cost * 2/3

35837910.0

In [28]:
project._phases['ExportSystemDesign'].total_length*2/3

60.03

#### Offshore Substation Design

In [29]:
project._phases["OffshoreSubstationDesign"].total_phase_cost

KeyError: 'OffshoreSubstationDesign'

In [ ]:
project._phases["OffshoreSubstationDesign"].design_result

#### Monopile Design

In [ ]:
# config['monopile']['weight'] * config['plant']['num_turbines'] * 3000 <<<<------ OLD


In [ ]:
project._phases["MonopileDesign"].total_phase_cost # <<<---- NEW

In [ ]:
project._phases["MonopileDesign"].design_result


#### Scour Protection Design

In [ ]:
project._phases['ScourProtectionDesign'].design_result  # <--- NEW, revised protection depth

In [ ]:
project._phases['ScourProtectionDesign'].protection_depth

In [ ]:
project._phases['ScourProtectionDesign'].total_phase_cost # <---- NEW

#### Monopile Installation

In [ ]:
project._phases['MonopileInstallation'].total_phase_time / (8760 / 12)

In [ ]:
project._phases['MonopileInstallation'].total_phase_cost

In [ ]:
project._phases['MonopileInstallation'].detailed_output

In [ ]:
project._phases['MonopileInstallation'].total_phase_time / 67

In [ ]:
mi = project.project_dataframe.loc[project.project_dataframe['phase']=='MonopileInstallation']

mi.groupby('action').sum()['duration']

#### Scour Protection Installation

In [ ]:
project._phases['ScourProtectionInstallation'].total_phase_time / (8760 / 12) # <--- NEW, revised protection depth

In [ ]:
project._phases['ScourProtectionInstallation'].total_phase_cost # <--- NEW, revised protection depth

In [ ]:
project._phases['ScourProtectionInstallation'].detailed_output

In [ ]:
project._phases['ScourProtectionInstallation'].total_phase_time / 67  # <--- NEW, revised protection depth

In [ ]:
spi = project.project_dataframe.loc[project.project_dataframe['phase']=='ScourProtectionInstallation']

spi.groupby('action').sum()['duration']

#### Turbine Installation

In [ ]:
project._phases['TurbineInstallation'].total_phase_time / (8760 / 12)  # NEW, turbine installation revised to eliminate too many crane reequips between blade installations

In [ ]:
project._phases['TurbineInstallation'].total_phase_cost  # NEW, see above

In [ ]:
project._phases['TurbineInstallation'].detailed_output

In [ ]:
project._phases['TurbineInstallation'].total_phase_time / 67  # NEW, see above

In [ ]:
ti = project.project_dataframe.loc[project.project_dataframe['phase']=='TurbineInstallation']

ti.groupby('action').sum()['duration']

In [ ]:
project._phases['TurbineInstallation'].wtiv.trip_data

#### Array System Installation

In [ ]:
project._phases['ArrayCableInstallation'].total_phase_time / (8760 / 12)  # NEW, not sure what changed but Rob should know. Burial rates?

In [ ]:
project._phases['ArrayCableInstallation'].total_phase_cost

In [ ]:
project._phases['ArrayCableInstallation'].detailed_output

In [ ]:
asi = project.project_dataframe.loc[project.project_dataframe['phase']=='ArrayCableInstallation']

asi.groupby('action').sum()['duration']

#### Export System Installation

In [ ]:
project._phases['ExportCableInstallation'].total_phase_time # NEW, not sure what changed but Rob should know. Burial rates?

In [ ]:
project._phases['ExportCableInstallation'].total_phase_cost # NEW, not sure what changed but Rob should know. Burial rates?

In [ ]:
project._phases['ExportCableInstallation'].detailed_output

In [ ]:
esi = project.project_dataframe.loc[project.project_dataframe['phase']=='ExportCableInstallation']

esi.groupby('action').sum()['duration']

#### Offshore Substation Installation

In [ ]:
project._phases['OffshoreSubstationInstallation'].total_phase_time / (24)

In [ ]:
project._phases['OffshoreSubstationInstallation'].total_phase_cost

In [ ]:
project._phases['OffshoreSubstationInstallation'].detailed_output

In [ ]:
osi = project.project_dataframe.loc[project.project_dataframe['phase']=='OffshoreSubstationInstallation']

osi.groupby('action').sum()['duration']

In [ ]:
project.project_dataframe["phase"].unique()

In [ ]:
project.design_results

In [ ]:
project.project_dataframe.time.max() / (8760 / 12)

In [ ]:
fig = plt.figure(figsize=(6, 4), dpi=200)
axis = fig.add_subplot(111)

pd.Series(project.phase_costs).plot(kind='bar', ax=axis)

axis.set_ylabel('Phase Cost ($USD)')

In [ ]:
fig = plt.figure(figsize=(6, 4), dpi=200)
axis = fig.add_subplot(111)

pd.Series(project.phase_times).plot(kind='bar', ax=axis)

axis.set_ylabel('Phase Time (h)')

In [ ]:
phases = project.phase_dates.keys()
_start_dates = [project.phase_dates[p]['start'] for p in phases]
start_floats = [plt_dates.datestr2num(i) for i in _start_dates]
_end_dates = [project.phase_dates[p]['end'] for p in phases]
end_floats = [plt_dates.datestr2num(i) for i in _end_dates]

In [ ]:
data = pd.DataFrame(columns=phases, data=[start_floats, end_floats], index=['Start', 'End']).T
data = data.reset_index().rename(columns={'index': 'Phase'})
data['Type'] = 'Model'

validation_data = data.copy()
validation_data = validation_data.set_index(keys='Phase')
validation_data.loc['MonopileInstallation', 'End'] = plt_dates.datestr2num("08/03/16 00:00")
validation_data.loc['ScourProtectionInstallation', 'End'] = plt_dates.datestr2num("09/30/16 00:00")
validation_data.loc['TurbineInstallation', 'End'] = plt_dates.datestr2num("09/04/17 00:00")
validation_data.loc['ArrayCableInstallation', 'End'] = plt_dates.datestr2num("08/27/16 00:00")
validation_data.loc['ExportCableInstallation', 'End'] = plt_dates.datestr2num("04/21/16 00:00")
validation_data['Type'] = 'Dudgeon'
validation_data = validation_data.reset_index().rename(columns={'index': 'Phase'})

data = pd.concat([data, validation_data])
data['Time'] = data['End'] - data['Start']
data

In [ ]:
import seaborn as sns
fig = plt.figure(figsize=(6, 4), dpi=200)
axis = fig.add_subplot(111)

axis = sns.barplot(x='Phase', y='Time', hue='Type', data=data, ax=axis)

for tick in axis.get_xticklabels():
    tick.set_rotation(90)
    
# axis.yaxis_date()
# fig.autofmt_xdate()

In [ ]:
fig = plt.figure(figsize=(6, 4), dpi=200)
axis = fig.add_subplot(111)

axis = sns.barplot(x='End', y='Phase', data=data, ax=axis, zorder=3, hue='Type')
axis = sns.barplot(x='Start', y='Phase', data=data, ax=axis, palette={'Model': 'w', 'Dudgeon': 'w'}, zorder=5, hue='Type')

axis.set_xlim(data['Start'].min()-10, data['End'].max()+10)

# Legend
handles, labels = axis.get_legend_handles_labels()
axis.legend(handles[:2], labels[:2], fontsize=8)

# Formatting
axis.set_xlabel('')
# axis.grid(axis='x', which='major', lw=0.5, zorder=4)
# Tick parameters
axis.tick_params(axis='both', labelsize=8, width=0.5, length=3)

# Plot formatting
for ax in ['top', 'bottom', 'left', 'right']:
    axis.spines[ax].set_linewidth(0.5)
    axis.spines[ax].set_zorder(20)

axis.xaxis_date()
fig.autofmt_xdate()